In [44]:
customer_order = """醤油ラーメンを3つ、うち1つは大盛にして下さい。
あとビールの大ジョッキを3つください。
"""

order_template = """以下の注文情報に基づいて以下の情報を抽出して下さい。
order_item: 種類ごとの商品名(item_name)とその個数(item_count)

出力JSONはorder_itemのリストで、order_itemの要素は以下のような形式です。
order_item:{{item_name:<商品名>,item_count:<個数>}}

注文情報: {order}
"""

In [45]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

order_item_schema = ResponseSchema(name="order_item",
                             description="商品名と個数をもった注文情報")
item_name_schema = ResponseSchema(name="item_name",
                             description="商品名、大盛と普通盛は別種類とする")
item_count_schema = ResponseSchema(name="item_count",
                                      description="商品の個数、指定が無い場合は1とする")

response_schemas = [order_item_schema, item_name_schema, item_count_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [46]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"order_item": string  // 商品名と個数をもった注文情報
	"item_name": string  // 商品名、大盛と普通盛は別種類とする
	"item_count": string  // 商品の個数、指定が無い場合は1とする
}
```


In [47]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=order_template)

messages = prompt.format_messages(order=customer_order, 
                                format_instructions=format_instructions)

In [48]:
print(messages[0].content)

以下の注文情報に基づいて以下の情報を抽出して下さい。
order_item: 種類ごとの商品名(item_name)とその個数(item_count)

出力JSONはorder_itemのリストで、order_itemの要素は以下のような形式です。
order_item:
    item_name
    item_count

注文情報: 醤油ラーメンを3つ、うち1つは大盛にして下さい。
あとビールの大ジョッキを3つください。




In [49]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    openai_api_base=os.getenv('AOAI_ENDPOINT'),
    openai_api_version="2023-05-15",
    deployment_name=os.getenv('AOAI_MODEL'),
    openai_api_key=os.getenv('AOAI_API_KEY'),
    openai_api_type = "azure",
    temperature=0.0,
)
response = chat(messages)

In [50]:
print(response.content)

[
  {
    "item_name": "醤油ラーメン",
    "item_count": 2
  },
  {
    "item_name": "大盛醤油ラーメン",
    "item_count": 1
  },
  {
    "item_name": "ビールの大ジョッキ",
    "item_count": 3
  }
]


In [51]:
output_dict = output_parser.parse(response.content)

OutputParserException: Got invalid return object. Expected key `order_item` to be present, but got [{'item_name': '醤油ラーメン', 'item_count': 2}, {'item_name': '大盛醤油ラーメン', 'item_count': 1}, {'item_name': 'ビールの大ジョッキ', 'item_count': 3}]